In [24]:
from pydub import AudioSegment
import os

# Create the directory if it doesn't exist
os.makedirs("../../Dataset/data", exist_ok=True)
# Path to your MP3 file
mp3_file_path = "../../Dataset/SandalWoodNewsStories_41.mp3"
# Path where you want to save the WAV file
wav_file_path = "../../Dataset/data/SandalWoodNewsStories_41.wav"

# Load the MP3 file
audio = AudioSegment.from_mp3(mp3_file_path)

# Export as WAV
audio.export(wav_file_path, format="wav")

print("Conversion complete!")

Conversion complete!


In [26]:
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import librosa

# Load audio
audio_path = "../../Dataset/data/SandalWoodNewsStories_41.wav"
audio, sr = librosa.load(audio_path, sr=16000)

# Load model and processor
model_id = "vasista22/whisper-kannada-tiny"
processor = WhisperProcessor.from_pretrained(model_id)
model = WhisperForConditionalGeneration.from_pretrained(model_id)
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(
    language="kn", task="transcribe"
)


# Process in smaller chunks
def process_audio(audio_array, chunk_duration=10, sr=16000):
    chunk_length = chunk_duration * sr
    transcription = []

    for i in range(0, len(audio_array), chunk_length):
        # Get chunk
        chunk = audio_array[i : i + chunk_length]

        # Process audio
        input_features = processor(
            chunk, sampling_rate=sr, return_tensors="pt"
        ).input_features

        # Generate tokens
        predicted_ids = model.generate(input_features)

        # Decode
        transcribed_text = processor.batch_decode(
            predicted_ids, skip_special_tokens=True
        )[0]

        transcription.append(transcribed_text)
        print(f"Processed chunk {i//chunk_length + 1}")

    return " ".join(transcription)


try:
    # Process the audio
    result = process_audio(audio)
    print("\nFull Transcription:", result)

except Exception as e:
    print(f"An error occurred: {str(e)}")
    print(f"Device being used: {device}")
    print(f"Model name: {model_id}")

An error occurred: index -2 is out of bounds for dimension 0 with size 0
Device being used: cpu
Model name: vasista22/whisper-kannada-tiny
